<a href="https://colab.research.google.com/github/juanmed/dw_a/blob/main/DLL_package_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --pre tensorflow=="2.2.0"

     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 460kB 41.8MB/s 
     |████████████████████████████████| 3.0MB 42.0MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [2]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2449, done.
remote: Counting objects: 100% (2449/2449), done.
remote: Compressing objects: 100% (2044/2044), done.
remote: Total 2449 (delta 593), reused 1357 (delta 378), pack-reused 0
Receiving objects: 100% (2449/2449), 30.84 MiB | 17.73 MiB/s, done.
Resolving deltas: 100% (593/593), done.


In [3]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1624302 sha256=2a0ddeefebf1f12a257c3f9eec512d752ce23ebd0ff06a5b11c949f0979b6803
  Stored in directory: /tmp/pip-ephem-wheel-cache-8hapxnen/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.1-cp37-none-any.whl size=43735 sha256=180829f6eb7e2d688ff91ee8d62203ed3070c1d1b90eee93849733a28a510478
  Stored in directory: /root/.cache/pip/wheels/65/fe/90/20d6d6d97223d80d20cb390be636619c536edab5658c12bdba
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=21cda8f528249a5655190c8a968afef9688bdd23f8e754ac974b48f888ad61db
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=90183fe798c31797ad142c85eef8cc8be364161d8a27359dc26a

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.1 which is incompatible.


In [4]:
import pathlib as pl

training_path = "./workspace/training_demo"
annotations_path = "./annotations"

if not os.path.exists(training_path+"/images/train"):
  pl.Path(training_path+"/images/train").mkdir(parents=True, exist_ok=True)
if not os.path.exists(training_path+"/test"):
  pl.Path(training_path+"/images/test").mkdir(parents=True, exist_ok=True)
if not os.path.exists(annotations_path):
  pl.Path(annotations_path).mkdir(parents=True, exist_ok=True)

In [5]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=e1c9a764895a1491cdbde19c022955852d880d408a9d8fea3b10f31c4efcb920
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [6]:
import wget
dataset_partition_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/dataset_partition.py"
dataset_partition_file = wget.download(dataset_partition_url)

In [14]:
# Partition the dataset
%%bash
python dataset_partition.py -x -i "./workspace/training_demo/images" -r 0.1

In [8]:
gen_tf_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/generate_tfrecord.py"
wget.download(gen_tf_url)

'generate_tfrecord.py'

In [9]:
import shutil
labelmap_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/label_map.pbtxt"
labelmap_file = wget.download(labelmap_url)

anno_path = "./workspace/training_demo/annotations"
if not os.path.exists(anno_path):
  pl.Path(anno_path).mkdir(parents=True, exist_ok=True)

shutil.move(labelmap_file, anno_path+"/"+labelmap_file)

'./workspace/training_demo/annotations/label_map.pbtxt'

In [15]:
# Generate .record files
%%bash
python generate_tfrecord.py -x ./workspace/training_demo/images/train -l ./workspace/training_demo/annotations/label_map.pbtxt -o ./workspace/training_demo/annotations/train.record
python generate_tfrecord.py -x ./workspace/training_demo/images/test -l ./workspace/training_demo/annotations/label_map.pbtxt -o ./workspace/training_demo/annotations/test.record

Successfully created the TFRecord file: ./workspace/training_demo/annotations/train.record
Successfully created the TFRecord file: ./workspace/training_demo/annotations/test.record


In [11]:
# Download pretrained EfficienteNet D5 1280x1280 from
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md


In [12]:

import sys
effnet_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d5_coco17_tpu-32.tar.gz"
filename = wget.download(effnet_url)

In [13]:
import tarfile

pretrained_path = "./workspace/training_demo/pre-trained-models"
if not os.path.exists(pretrained_path):
  pl.Path(pretrained_path).mkdir(parents=True, exist_ok=True)

tar = tarfile.open(filename) 
tar.extractall(path=pretrained_path) # untar file into same directory
tar.close()

In [23]:
# Configure model
models_path = "./workspace/training_demo/models/efficientdet_d5_coco17_tpu-32"
if not os.path.exists(models_path):
  pl.Path(models_path).mkdir(parents=True, exist_ok=True)

config_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/pipeline.config"
config_file = wget.download(config_url)

shutil.move(config_file, models_path + "/" + config_file)
shutil.copy("./models/research/object_detection/model_main_tf2.py","./workspace/training_demo/model_main_tf2.py")

'./workspace/training_demo/model_main_tf2.py'

In [25]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [26]:
# Start training
%%bash
cd workspace/training_demo/
python model_main_tf2.py --model_dir=models/efficientdet_d5_coco17_tpu-32 --pipeline_config_path=models/efficientdet_d5_coco17_tpu-32/pipeline.config

2021-03-05 21:55:10.110680: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
W0305 21:55:10.146646 139873799792512 cross_device_ops.py:1321] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0305 21:55:10.147032 139873799792512 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0305 21:55:10.156262 139873799792512 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0305 21:55:10.156512 139873799792512 config_util.py:552] Maybe overwriting use_bfloat16: False
I0305 21:55:10.182673 139873799792512 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b

#Test

In [17]:
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

In [18]:
import tensorflow as tf
tf.keras.backend.clear_session()
print('Building model and restoring weights for fine-tuning...', flush=True)
num_classes = 1
pipeline_config = models_path + "/" + config_file
checkpoint_path = './workspace/training_demo/pre-trained-models/efficientdet_d5_coco17_tpu-32/checkpoint/ckpt-0'

# Load pipeline config and build a detection model.
#
# Since we are working off of a COCO architecture which predicts 90
# class slots by default, we override the `num_classes` field here to be just
# one (for our new rubber ducky class).
num_classes = 5
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True
detection_model = model_builder.build(
      model_config=model_config, is_training=True)

Building model and restoring weights for fine-tuning...


In [19]:
# Set up object-based checkpoint restore --- RetinaNet has two prediction
# `heads` --- one for classification, the other for box regression.  We will
# restore the box regression head but initialize the classification head
# from scratch (we show the omission below by commenting out the line that
# we would add if we wanted to restore both heads)
fake_box_predictor = tf.compat.v2.train.Checkpoint(
    _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
    # _prediction_heads=detection_model._box_predictor._prediction_heads,
    #    (i.e., the classification head that we *will not* restore)
    _box_prediction_head=detection_model._box_predictor._box_prediction_head,
    )
fake_model = tf.compat.v2.train.Checkpoint(
          _feature_extractor=detection_model._feature_extractor,
          _box_predictor=fake_box_predictor)
ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
ckpt.restore(checkpoint_path).expect_partial()

# Run model through a dummy image so that variables are created
image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = detection_model.predict(image, shapes)
_ = detection_model.postprocess(prediction_dict, shapes)
print('Weights restored!')

Weights restored!


In [20]:
tf.keras.backend.set_learning_phase(True)
# These parameters can be tuned; since our training set has 5 images
# it doesn't make sense to have a much larger batch size, though we could
# fit more examples in memory if we wanted to.
batch_size = 4
learning_rate = 0.01
num_batches = 100

# Select variables in top layers to fine-tune.
trainable_variables = detection_model.trainable_variables
to_fine_tune = []
prefixes_to_train = [
  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']
for var in trainable_variables:
  if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
    to_fine_tune.append(var)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [21]:
# Set up forward + backward pass for a single train step.
def get_model_train_step_function(model, optimizer, vars_to_fine_tune):
  """Get a tf.function for training step."""

  # Use tf.function for a bit of speed.
  # Comment out the tf.function decorator if you want the inside of the
  # function to run eagerly.
  @tf.function
  def train_step_fn(image_tensors,
                    groundtruth_boxes_list,
                    groundtruth_classes_list):
    """A single training iteration.

    Args:
      image_tensors: A list of [1, height, width, 3] Tensor of type tf.float32.
        Note that the height and width can vary across images, as they are
        reshaped within this function to be 640x640.
      groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
        tf.float32 representing groundtruth boxes for each image in the batch.
      groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
        with type tf.float32 representing groundtruth boxes for each image in
        the batch.

    Returns:
      A scalar tensor representing the total loss for the input batch.
    """
    shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
    model.provide_groundtruth(
        groundtruth_boxes_list=groundtruth_boxes_list,
        groundtruth_classes_list=groundtruth_classes_list)
    with tf.GradientTape() as tape:
      preprocessed_images = tf.concat(
          [detection_model.preprocess(image_tensor)[0]
           for image_tensor in image_tensors], axis=0)
      prediction_dict = model.predict(preprocessed_images, shapes)
      losses_dict = model.loss(prediction_dict, shapes)
      total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
      gradients = tape.gradient(total_loss, vars_to_fine_tune)
      optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
    return total_loss

  return train_step_fn

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
train_step_fn = get_model_train_step_function(
    detection_model, optimizer, to_fine_tune)


In [22]:
print('Start fine-tuning!', flush=True)
for idx in range(num_batches):
  # Grab keys for a random subset of examples
  all_keys = list(range(len(train_images_np)))
  random.shuffle(all_keys)
  example_keys = all_keys[:batch_size]

  # Note that we do not do data augmentation in this demo.  If you want a
  # a fun exercise, we recommend experimenting with random horizontal flipping
  # and random cropping :)
  gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
  gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
  image_tensors = [train_image_tensors[key] for key in example_keys]

  # Training step (forward pass + backwards pass)
  total_loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list)

  if idx % 10 == 0:
    print('batch ' + str(idx) + ' of ' + str(num_batches)
    + ', loss=' +  str(total_loss.numpy()), flush=True)

print('Done fine-tuning!')

Start fine-tuning!


NameError: ignored